In [1]:
import os
import re

import numpy as np

from numpy import array
from keras.models import Sequential , Model
from keras.layers import Dense ,Input 
from keras.layers import TimeDistributed
from keras.layers import LSTM
from keras.layers import merge,Dot,Reshape,RepeatVector

Using TensorFlow backend.


### TimeDistributed and return_sequence

https://datascience.stackexchange.com/questions/10836/the-difference-between-dense-and-timedistributeddense-of-keras

In [9]:
# prepare sequence
length = 5
seq = array([i/float(length) for i in range(length*4)])
X = seq.reshape(-1, length, 4)
y = seq.reshape(-1, length, 4)
# define LSTM configuration
n_neurons = 3#length

x = Input(shape = (length,4))
out = LSTM(n_neurons,  return_sequences=False,return_state=False)(x)
out = Reshape((1,3),input_shape=(3,))(out)
#out = TimeDistributed(Dense(1))(out)
model = Model(inputs=x , outputs=out)
model.summary()

model.predict(X)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 5, 4)              0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 3)                 96        
_________________________________________________________________
reshape_1 (Reshape)          (None, 1, 3)              0         
Total params: 96
Trainable params: 96
Non-trainable params: 0
_________________________________________________________________


array([[[ 0.27481663, -0.85328579, -0.2766172 ]]], dtype=float32)

In [4]:
# prepare sequence
length = 5
seq = array([i/float(length) for i in range(length*4)])
X = seq.reshape(-1, length, 4)
y = seq.reshape(-1, length, 4)
# define LSTM configuration
n_neurons = 3#length

x = Input(shape = (length,4))
out = LSTM(n_neurons,  return_sequences=False,return_state=False)(x)
#out = TimeDistributed(Dense(1))(out)
model = Model(inputs=x , outputs=out)
model.summary()

model.predict(X)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 5, 4)              0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 3)                 96        
Total params: 96
Trainable params: 96
Non-trainable params: 0
_________________________________________________________________


array([[-0.        ,  0.50033331, -0.78666264]], dtype=float32)

In [4]:
# prepare sequence
length = 5
seq = array([i/float(length) for i in range(length*4)])
X = seq.reshape(-1, length, 4)
y = seq.reshape(-1, length, 4)
# define LSTM configuration
n_neurons = 3#length

x = Input(shape = (length,4))
out = LSTM(n_neurons,  return_sequences=True,return_state=False)(x)
out = TimeDistributed(Dense(1))(out)
model = Model(inputs=x , outputs=out)
model.summary()

model.predict(X)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 5, 4)              0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 5, 3)              96        
_________________________________________________________________
time_distributed_1 (TimeDist (None, 5, 1)              4         
Total params: 100
Trainable params: 100
Non-trainable params: 0
_________________________________________________________________


array([[[ 0.09903315],
        [ 0.21050203],
        [ 0.28685269],
        [ 0.36754459],
        [ 0.46470472]]], dtype=float32)

In [5]:
from keras.layers import Lambda, Input
from keras.models import Model
#from keras.backend import softmax #無法設定axis
from keras.activations import softmax #可設定axis
import numpy as np

inputo = Input(shape=(5,1))
double = Lambda(lambda x: softmax(x , axis=1))(inputo)

model = Model(inputs=[inputo], outputs=[double])
model.compile(optimizer='sgd', loss='mse')
model.summary()
data = np.arange(5).reshape((-1,5,1))
print(model.predict(data))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 5, 1)              0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 5, 1)              0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________
[[[ 0.01165623]
  [ 0.03168492]
  [ 0.08612854]
  [ 0.23412167]
  [ 0.63640863]]]


In [6]:
data

array([[[0],
        [1],
        [2],
        [3],
        [4]]])

In [7]:
x = data.reshape(-1)    

np.exp(x)/sum(np.exp(x))

array([ 0.01165623,  0.03168492,  0.08612854,  0.23412166,  0.63640865])

## Attention Layer

In [7]:
from keras.layers import Multiply
from keras.layers import Lambda, Input
from keras.models import Model
#from keras.backend import softmax #無法設定axis
from keras.activations import softmax #可設定axis
import numpy as np

length=2
my_input = Input(shape=(length,5))
ls = LSTM(10,return_sequences=True)(my_input)

td = TimeDistributed(Dense(1,activation='linear',use_bias=False))(ls)
sm = Lambda(lambda x : softmax(x,axis=1) , name='softmax')(td)

attn = Multiply()([sm,ls])

out = LSTM(5,return_sequences=True,name="hi")(attn)

dot_out = Multiply()([out,my_input])
dot_out = TimeDistributed(Dense(1,activation='linear',kernel_initializer='ones',use_bias=False,trainable=False))(dot_out)

model = Model(inputs=[my_input] , outputs=[out])
model.compile(loss = 'mse',optimizer='adam')
model.summary()

model_1 = Model(inputs=[my_input] , outputs=[ls])
model_2 = Model(inputs=[my_input] , outputs=[sm])
model_3 = Model(inputs=[my_input] , outputs=[attn])
model_4 = Model(inputs=[my_input] , outputs=[dot_out])

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_6 (InputLayer)             (None, 2, 5)          0                                            
____________________________________________________________________________________________________
lstm_10 (LSTM)                   (None, 2, 10)         640         input_6[0][0]                    
____________________________________________________________________________________________________
time_distributed_11 (TimeDistrib (None, 2, 1)          10          lstm_10[0][0]                    
____________________________________________________________________________________________________
softmax (Lambda)                 (None, 2, 1)          0           time_distributed_11[0][0]        
___________________________________________________________________________________________

In [17]:
x = np.arange(10).reshape((-1,length,5))/10
y = x.reshape((-1))[::-1].reshape((-1,length,5))
#model.fit(x,y,epochs=1000,verbose=0)
model.predict(x,batch_size=1)

array([[[ 0.00140486,  0.00721304,  0.00440872,  0.00188392, -0.00086394],
        [-0.00080154,  0.01967781,  0.00544048, -0.0004214 , -0.00863076]]], dtype=float32)

In [18]:
model.compile(loss='mse',metrics=['mse'],optimizer='adam')
model.fit(x,y,epochs=10)

Epoch 1/10
1/1 [==============================] - 0s - loss: 0.2816 - mean_squared_error: 0.2816
Epoch 2/10
1/1 [==============================] - 0s - loss: 0.2813 - mean_squared_error: 0.2813
Epoch 3/10
1/1 [==============================] - 0s - loss: 0.2809 - mean_squared_error: 0.2809
Epoch 4/10
1/1 [==============================] - 0s - loss: 0.2806 - mean_squared_error: 0.2806
Epoch 5/10
1/1 [==============================] - 0s - loss: 0.2802 - mean_squared_error: 0.2802
Epoch 6/10
1/1 [==============================] - 0s - loss: 0.2798 - mean_squared_error: 0.2798
Epoch 7/10
1/1 [==============================] - 0s - loss: 0.2795 - mean_squared_error: 0.2795
Epoch 8/10
1/1 [==============================] - 0s - loss: 0.2791 - mean_squared_error: 0.2791
Epoch 9/10
1/1 [==============================] - 0s - loss: 0.2787 - mean_squared_error: 0.2787
Epoch 10/10
1/1 [==============================] - 0s - loss: 0.2783 - mean_squared_error: 0.2783


In [10]:
a1 = model_1.predict(x)
a2 = model_2.predict(x)
a3 = model_3.predict(x)

a1.shape,a2.shape,a3.shape

((1, 2, 10), (1, 2, 1), (1, 2, 10))

In [11]:
(a1[0][1]*a2[0][1]==a3[0][1]).all()

True

In [30]:
## dot product

my_input = Input(shape = (1,5))
out = Input(shape = (1,5))

dot_out = Multiply()([out,my_input])
dot_out = TimeDistributed(Dense(1,activation='linear',kernel_initializer='ones',use_bias=False,trainable=False))(dot_out)

model_4=Model(inputs=[my_input,out],outputs=[dot_out])
model_4.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_22 (InputLayer)            (None, 1, 5)          0                                            
____________________________________________________________________________________________________
input_21 (InputLayer)            (None, 1, 5)          0                                            
____________________________________________________________________________________________________
multiply_21 (Multiply)           (None, 1, 5)          0           input_22[0][0]                   
                                                                   input_21[0][0]                   
____________________________________________________________________________________________________
time_distributed_21 (TimeDistrib (None, 1, 1)          5           multiply_21[0][0]       

In [33]:
x1 = np.array([[1,2,3,4,5]]).reshape((1,1,5))
x2 = np.array([[1,2,1,1,1]]).reshape((1,1,5))
model_4.predict([x1,x2])

array([[[ 17.]]], dtype=float32)